In [ ]:
! pip install datasets transformers nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 77.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 35.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 2.7 MB/s eta 0:00:00


In [ ]:
from transformers import (AutoModelForSeq2SeqLM, 
                            AutoTokenizer, 
                            T5Tokenizer)
import torch
import pandas as pd
from datasets import Dataset, DatasetDict, load_dataset, load_from_disk
from tqdm import tqdm

#import subprocess
import sys
import os
import argparse
from IPython.core import error
import random
import numpy as np
import nltk
import json
import csv

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
output_dir = 'path_to_your_model_weight/'
# Here you need to link this path in your Google drive to the place preseving your model weights, e.g., checkpoint-62500
# You can download it on the github page

model_checkpoint = "t5-base"
prefix = "Transform the following sentence into Signal Temporal logic: "

max_input_length = 1024
max_target_length = 128
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, model_max_length=max_input_length)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
tl_model = AutoModelForSeq2SeqLM.from_pretrained(output_dir+"checkpoint-62500").to(device)

In [ ]:
import time
time_start = time.time()
inputs = [prefix + 'At some point (prop_1), and at some point (prop_2), and always do not (prop_4).']
inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, return_tensors="pt").to(device)
output = tl_model.generate(**inputs, num_beams=8, do_sample=True, max_length=max_target_length)
decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
print(decoded_output)
time_end = time.time()
print('Translation time: ', time_end-time_start)

( ( prop_1 and prop_2 ) and globally ( negation prop_4 ) )
Translation time:  4.9763360023498535


In [6]:
# Here are the example test sentences
test_sentence = ['Stay at (prop_1) for 5 units in the future and stay at (prop_2) for 5 units in the future, and ensure that never (prop_3).',
                 'First (prop_1), and then (prop_2), and ensure that never (prop_3).',
                 'Start by (prop_1). Then, (prop_2). Lastly, (prop_3).',
                 'Guarantee that you (prop_1) and (prop_2)', # Input the natural sentence
                 '( prop_1 ) and whenever ( prop_2 )',
                 'Sooner or later (prop_1)',
                 'Repeatedly (prop_1)',
                 'At some point, (prop_1).',
                 'Do prop_1 but not do prop_2',
                 'Do prop_1, do prop_2, do prop_3'] # Input the natural sentence

for sentence in test_sentence:
  inputs = [prefix + sentence]
  inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, return_tensors="pt").to(device)
  output = tl_model.generate(**inputs, num_beams=8, do_sample=True, max_length=max_target_length)
  decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
  print('Input sentence: ', sentence)
  print('Translated STL: ', decoded_output)
  print('\n')

Input sentence:  Stay at (prop_1) for 5 units in the future and stay at (prop_2) for 5 units in the future, and ensure that never (prop_3).
Translated STL:  ( ( globally [0,5] prop_1 and globally [0,5] prop_2 ) and globally [0,1] negation prop_3 )


Input sentence:  First (prop_1), and then (prop_2), and ensure that never (prop_3).
Translated STL:  ( ( prop_1 until prop_2 ) and globally ( negation prop_3 ) )


Input sentence:  Start by (prop_1). Then, (prop_2). Lastly, (prop_3).
Translated STL:  ( ( prop_1 until prop_2 ) until prop_3 )


Input sentence:  Guarantee that you (prop_1) and (prop_2)
Translated STL:  finally ( prop_1 and prop_2 )


Input sentence:  ( prop_1 ) and whenever ( prop_2 )
Translated STL:  ( prop_1 and finally prop_2 )


Input sentence:  Sooner or later (prop_1)
Translated STL:  finally prop_1


Input sentence:  Repeatedly (prop_1)
Translated STL:  globally ( finally prop_1 )


Input sentence:  At some point, (prop_1).
Translated STL:  finally prop_1


Input senten